In [1]:
import numpy as np
import tensorflow as tf

In [56]:
class MLP:
    def __init__(self,data_path, model_path,save_path ,hidden_node_num):
        #Define instance variable
        self.__data_path = data_path
        self.__model_path = model_path
        self.__save_path = save_path
        self.__hidden_node_num = hidden_node_num
        self.__hidden_layer_num = len(self.__hidden_node_num)
        
        self.__data = None
        self.__y_data = None
        self.__data_len = None
        
        self.__y = None
        self.__x = None
        
        self.__w = []
        self.__b = []
        self.__layer = []
        
        self.__y_out = None
        self.__train = None
        self.__sess = None
        self.__saver = None
        
        self.__result = []
        
        self.__loadData()
        self.__init_sess()
        self.__save_result()
        
    def __loadData(self):
        tf.reset_default_graph()
                
        xy = np.genfromtxt(self.__data_path, dtype='float32')
        data_num = xy[:,1:-1].shape[0]
        self.__data =  xy[:,1:-1]
        self.__y_data =  xy[:,-1]
        self.__data = self.__data.transpose() 
        self.__data_len = len(self.__data)
        
        self.__x = tf.placeholder(dtype=tf.float32)
        
        # first layer
        self.__w.append(tf.Variable(tf.random_normal([self.__hidden_node_num[0], self.__data_len]), name="w0"))
        self.__b.append(tf.Variable(tf.random_normal([self.__hidden_node_num[0],1]), name="b0"))
       
        # add hidden layers (variable number)
        for i in range(1,self.__hidden_layer_num):
            wName = "w" + str(i)
            bName = "b" + str(i)
            self.__w.append(tf.Variable(tf.random_normal([self.__hidden_node_num[i], self.__hidden_node_num[i-1]]), name=wName))
            self.__b.append(tf.Variable(tf.random_normal([self.__hidden_node_num[i],1]), name=bName))
        
        # add final layer
        wName = "w" + str(self.__hidden_layer_num)
        bName = "b" + str(self.__hidden_layer_num)
        self.__w.append(tf.Variable(tf.random_normal([1, self.__hidden_node_num[-1]]), name=wName))
        self.__b.append(tf.Variable(tf.random_normal([1],1), name=bName))
        
        # define model
        self.__layer.append(tf.nn.sigmoid(tf.matmul( self.__w[0],self.__x) + self.__b[0]))
        for i in range(1,self.__hidden_layer_num):
            self.__layer.append(tf.nn.sigmoid(tf.matmul( self.__w[i],self.__layer[i-1]) + self.__b[i]))
        self.__y_out = tf.matmul(self.__w[-1],self.__layer[-1]) + self.__b[-1]
        
    def __init_sess(self):
        init = tf.global_variables_initializer()
        self.__sess = tf.Session()
        self.__sess.run(init)
        self.__saver = tf.train.Saver()
        self.__saver.restore(self.__sess, self.__model_path)
        self.__result = self.__sess.run(self.__y_out,feed_dict={self.__x: self.__data})[0]
        
    def __save_result(self):
        saveOpen = open(self.__save_path,'w')
        lens = len(self.__y_data)
        sum_acc=0
        for i in range(lens):
            if self.__y_data[i] != 0:
                acc = (self.__y_data[i] - self.__result[i]) *100 /self.__y_data[i]
            else:
                acc = 0
            sum_acc += acc
            saveOpen.write(str(self.__y_data[i]) + "\t" + str(self.__result[i]) + "\t" +str(acc) + "\n")
        sum_acc /= lens
        saveOpen.write(str(sum_acc))
        saveOpen.close()

In [57]:
hidden_node_num = [50,20]

alpha_num = 16
start_alpha = -5
end_alpha = 10


In [58]:
dataPath = "data/data_Alpha_"
savePath = "accuracy"
modelPath = "model/0226_1711/model_"
result = []
for i in range(start_alpha,end_alpha+1):
    modelPaths = modelPath + str(i)
    dataPaths = dataPath + str(i) +".txt"
    savePaths = savePath + str(i) +".txt"
    myMLP = MLP(dataPaths, modelPaths, savePaths,hidden_node_num)

INFO:tensorflow:Restoring parameters from model/0226_1711/model_-5
INFO:tensorflow:Restoring parameters from model/0226_1711/model_-4
INFO:tensorflow:Restoring parameters from model/0226_1711/model_-3
INFO:tensorflow:Restoring parameters from model/0226_1711/model_-2
INFO:tensorflow:Restoring parameters from model/0226_1711/model_-1
INFO:tensorflow:Restoring parameters from model/0226_1711/model_0
INFO:tensorflow:Restoring parameters from model/0226_1711/model_1
INFO:tensorflow:Restoring parameters from model/0226_1711/model_2
INFO:tensorflow:Restoring parameters from model/0226_1711/model_3
INFO:tensorflow:Restoring parameters from model/0226_1711/model_4
INFO:tensorflow:Restoring parameters from model/0226_1711/model_5
INFO:tensorflow:Restoring parameters from model/0226_1711/model_6
INFO:tensorflow:Restoring parameters from model/0226_1711/model_7
INFO:tensorflow:Restoring parameters from model/0226_1711/model_8
INFO:tensorflow:Restoring parameters from model/0226_1711/model_9
INFO: